test file for nlp packages that predict race to see acuuracy,the following are the packages using 

1. ethnicolr
The model uses US census data, the Florida voting registration data, and the Wikipedia data collected by Skiena and colleagues, to predict race and ethnicity based on first and last name or just the last name. 
The model provides 06 functions. 

Source: https://ethnicolr.readthedocs.io/ethnicolr.html#install  



The Prediction Process: 

owner ----> nlp ----> W: 70% NW: 30%    
block group ... ----> W: 80% NW: 20%
            .
            .
            .
            .
            and use
        min & max scale? 
          ex.  
            1 -----> 40

In [37]:
import pandas as pd

#load data fram with names here
#this is the data frame we will use
mergent_true_df = pd.read_csv('../data/mergent_and_library/final_data_v2.csv')

mergent_true_df
#mergent_true_df.columns #ask about search column, what does this mean 

,Unnamed: 0,duns,company_name,firstname,lastname,title,gender,flag_owner,flag_presi_vicepresi,prob_nw_eth_wiki,...,primary_naics,naics2,naics_name,flag_executive_reported,geoid,prop_nwh,geometry,prob_spacy,prob_gt,prob_ld
0,0,00-138-3244,"WILSON, LYNNFORD SAMUEL JR",Lynnford,Wilson,Owner,U,1,0,0.917946,...,621111,62,Health Care and Social Assistance,1,5.105948e+10,26.602177,"c(-77.324596, 38.999535)",0.999995,0.0,1
1,1,00-145-3948,CROCODILE CONSULTING,Jessica,Moore,Administrative Assistant Sales,FEMALE,0,0,NaN,...,541611,54,"Professional, Scientific, and Technical Services",1,5.105948e+10,20.372268,"c(-77.389534, 38.994397)",0.000000,0.0,0
2,2,00-145-3948,CROCODILE CONSULTING,Leonard,Caldwell,Owner,MALE,1,0,NaN,...,541611,54,"Professional, Scientific, and Technical Services",1,5.105948e+10,20.372268,"c(-77.389534, 38.994397)",0.000000,0.0,0
3,3,00-171-0412,CENTREVILLE TAX SERVICE LLC,Scott,Walls,Owner,MALE,1,0,0.087375,...,541213,54,"Professional, Scientific, and Technical Services",1,5.105949e+10,19.153605,"c(-77.460635, 38.8639)",0.999994,0.0,1
4,4,00-183-0152,KAUFMAN ASSOC,Betsy,Kaufman,Owner,FEMALE,1,0,0.957856,...,541618,54,"Professional, Scientific, and Technical Services",1,5.105942e+10,36.208236,"c(-77.08656, 38.777228)",0.999673,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18151,18151,96-994-4219,"GEOPAY, INC.",Karla,Russell,Chief Operating Officer,FEMALE,0,0,0.747244,...,812990,81,Other Services (except Public Administration),1,5.105948e+10,34.302699,"c(-77.331479, 38.948677)",0.000000,0.0,0
18152,18152,96-995-4812,K EDWARD ROBINSON & ASSOCIATES LLC,Keith,Robinson,Pres-ceo,MALE,0,0,NaN,...,541611,54,"Professional, Scientific, and Technical Services",1,5.105942e+10,62.402930,"c(-77.230478, 38.707725)",0.999673,0.0,1
18153,18153,96-996-0488,"LAGNIAPPE CONSULTING, LLC",Wallace,Daniels,Owner,MALE,1,0,0.064103,...,541612,54,"Professional, Scientific, and Technical Services",1,5.105942e+10,60.324061,"c(-77.097715, 38.7433)",0.000000,0.0,0
18154,18154,96-996-0488,"LAGNIAPPE CONSULTING, LLC",Omar,Daniels,Co-owner,MALE,0,0,0.022954,...,541612,54,"Professional, Scientific, and Technical Services",1,5.105942e+10,60.324061,"c(-77.097715, 38.7433)",0.000000,0.0,0


# Package 1: Ethnicolr 
## (we are using 6 ethnicolr functions here)

In [38]:
import ethnicolr
from ethnicolr import census_ln, pred_census_ln

namesdf = pd.DataFrame(mergent_true_df, columns=['lastname', 'firstname'])
#the orignal data frame(mergent_executives_flagged.csv.xz) contains 29,876 names
#this is so i don't have to write it out
f = 'firstname'
l = 'lastname'
namesdf #dataframe with only first and last names

#-----------IMPORTANT----------
#1. for binary classification dfs and columns:
#I added a prop not_white column and a race column using below boolean values:
#1 if minority 
#0 if white

#2. I added a computation under each df to describe the number of minority guesses.. i also created
#a list of all the executive names that were flagged a minority if that helps in any way

,lastname,firstname
0,Wilson,Lynnford
1,Moore,Jessica
2,Caldwell,Leonard
3,Walls,Scott
4,Kaufman,Betsy
...,...,...
18151,Russell,Karla
18152,Robinson,Keith
18153,Daniels,Wallace
18154,Daniels,Omar


## pred_census_ln w firstnames

In [39]:
fdf = pred_census_ln(namesdf, f, 2010)

#sort to binary classification w first names

d = pd.DataFrame(fdf, columns=[f, 'white'])
d['not_white']= 1-d['white']


racelist = list(fdf['race'])
notw = list(d['not_white']) 
race = []

for n in notw:
    if n > .50:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)

race
d['race'] = race
d

#I added a prop not_white column and made it so that the race column used boolean values:
#1 if minority 
#0 if white

,firstname,white,not_white,race
0,Lynnford,0.772486,0.227514,0
1,Jessica,0.849624,0.150376,0
2,Leonard,0.881863,0.118137,0
3,Scott,0.537189,0.462811,0
4,Betsy,0.950811,0.049189,0
...,...,...,...,...
4526,Eben,0.978762,0.021238,0
4527,Thaksina,0.057127,0.942873,1
4528,Jeromy,0.391616,0.608384,1
4529,Maren,0.622192,0.377808,0


## pred_census_ln w lastnames

In [40]:
ldf = pred_census_ln(namesdf, l, 2010)
#sort into binary classification with last names

dd = pd.DataFrame(ldf, columns=[l, 'white'])
notw = list(1-dd['white'])
dd['not_white'] = notw

race = list(list(ldf['race']))

newlist = []
for r in race:
    newlist.append(str(r))
newlist

correct = []
for n in list(dd['not_white']):
    if .50 < n < 1.00:
        #print(True)
        correct.append(1)
    else:
        #print(False)  
        correct.append(0)


dd['race'] = correct
dd


,lastname,white,not_white,race
0,Wilson,0.700449,0.299551,0
1,Moore,0.734740,0.265260,0
2,Caldwell,0.712139,0.287861,0
3,Walls,0.765906,0.234094,0
4,Kaufman,0.971962,0.028038,0
...,...,...,...,...
3427,Monje,0.534677,0.465323,0
3428,Poon,0.362778,0.637222,1
3429,Montaquila,0.079733,0.920267,1
3430,Khaikaew,0.052132,0.947868,1


In [41]:
print(correct.count(1), len(race))

1049 3432


## pred_wiki_name (this takes both the first and last names)

In [42]:
from ethnicolr import pred_wiki_name
wikifulldf = pred_wiki_name(namesdf,l, f, conf_int=0.9)

white = ['GreaterEuropean,WestEuropean,Italian_mean', 'GreaterEuropean,WestEuropean,Nordic_mean', 
          'GreaterEuropean,WestEuropean,Germanic_mean', 'GreaterEuropean,WestEuropean,French_mean',
          'GreaterEuropean,Jewish_mean', 'GreaterEuropean,British_mean','GreaterEuropean,EastEuropean_mean']
whitelist = ['GreaterEuropean,WestEuropean,Italian', 'GreaterEuropean,WestEuropean,Nordic',
         'GreaterEuropean,WestEuropean,Germanic', 'GreaterEuropean,WestEuropean,French',
          'GreaterEuropean,Jewish', 'GreaterEuropean,British', 'GreaterEuropean,EastEuropean']

#sort into binary classification
a = pd.DataFrame(wikifulldf, columns=[f, l])
a['white'] = wikifulldf[white].sum(axis=1)
a['not_white'] = 1- a['white']
a['race'] = wikifulldf['race']

race = []

for n in list(a['not_white']):
    if .50 < n < 1.00:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)
 

a['race'] = race

a
#null values:
#defalut is 1.00 not_white and for thre race column thats automatcially gunna be 0

['Asian,GreaterEastAsian,EastAsian', 'Asian,GreaterEastAsian,Japanese', 'Asian,IndianSubContinent', 'GreaterAfrican,Africans', 'GreaterAfrican,Muslim', 'GreaterEuropean,British', 'GreaterEuropean,EastEuropean', 'GreaterEuropean,Jewish', 'GreaterEuropean,WestEuropean,French', 'GreaterEuropean,WestEuropean,Germanic', 'GreaterEuropean,WestEuropean,Hispanic', 'GreaterEuropean,WestEuropean,Italian', 'GreaterEuropean,WestEuropean,Nordic']


,firstname,lastname,white,not_white,race
0,Lynnford,Wilson,0.958743,0.041257,0
1,Jessica,Moore,0.804006,0.195994,0
2,Leonard,Caldwell,0.980906,0.019094,0
3,Scott,Walls,0.990065,0.009935,0
4,Betsy,Kaufman,0.957042,0.042958,0
...,...,...,...,...,...
3427,Arpuro,Monje,0.000000,1.000000,0
3428,Yuen,Poon,0.000000,1.000000,0
3429,Eben,Montaquila,0.000000,1.000000,0
3430,Thaksina,Khaikaew,0.000000,1.000000,0


In [43]:
print(race.count(1), len(race))

499 3432


##  pred_fl_reg_name(df, 'last', 'first', conf_int=0.9) w fullname

In [44]:
from ethnicolr import pred_fl_reg_name, pred_fl_reg_name_five_cat
odf = pred_fl_reg_name(namesdf, l, f, conf_int=0.9)

#sort binary
b = pd.DataFrame(odf, columns=[f, l])
b['white'] = odf['nh_white_mean']
b['not_white'] = 1 - b['white']
b['race'] = odf['race']


race = []

    
for n in list(b['not_white']):
    if n > .50:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)
 

b['race'] = race
b

['asian', 'hispanic', 'nh_black', 'nh_white']


,firstname,lastname,white,not_white,race
0,Lynnford,Wilson,0.594108,0.405892,0
1,Jessica,Moore,0.789375,0.210625,0
2,Leonard,Caldwell,0.710981,0.289019,0
3,Scott,Walls,0.971701,0.028299,0
4,Betsy,Kaufman,0.913341,0.086659,0
...,...,...,...,...,...
3427,Arpuro,Monje,NaN,NaN,0
3428,Yuen,Poon,NaN,NaN,0
3429,Eben,Montaquila,NaN,NaN,0
3430,Thaksina,Khaikaew,NaN,NaN,0


In [45]:
print(race.count(1), len(race))

485 3432


## pred_fl_reg_name_five_cat(df, namecol, num_iter=100, conf_int=1.0) w/full name input

In [49]:
rdf = pred_fl_reg_name_five_cat(namesdf, l, f, num_iter=100, conf_int=0.9)
#sort to binary
c = pd.DataFrame(rdf, columns=[f, l])
c['white'] = rdf['nh_white_mean']
c['not_white'] = 1 - c['white']
c['race'] = rdf['race']

racelist = list(c['race'])
race = []

        

for n in list(c['not_white']):
    if n > .50:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)
 

c['race'] = race
c

['asian', 'hispanic', 'nh_black', 'nh_white', 'other']


,firstname,lastname,white,not_white,race
0,Lynnford,Wilson,0.143666,0.856334,1
1,Jessica,Moore,0.421109,0.578891,1
2,Leonard,Caldwell,0.283802,0.716198,1
3,Scott,Walls,0.691393,0.308607,0
4,Betsy,Kaufman,0.417624,0.582376,1
...,...,...,...,...,...
3427,Arpuro,Monje,NaN,NaN,0
3428,Yuen,Poon,NaN,NaN,0
3429,Eben,Montaquila,NaN,NaN,0
3430,Thaksina,Khaikaew,NaN,NaN,0


In [50]:
print(race.count(1), len(race))

1049 3432


## pred_nc_reg_name(df, namecol, num_iter=100, conf_int=1.0) w/ full names 

In [51]:
from ethnicolr import pred_nc_reg_name
z = pred_nc_reg_name(namesdf, l, f, num_iter=100, conf_int=.90)
# / ***************************************************************************
# Race codes
# race               description
# *******************************************************************************
# A                  ASIAN
# B                  BLACK or AFRICAN AMERICAN
# I                  AMERICAN INDIAN or ALASKA NATIVE
# M                  TWO or MORE RACES
# O                  OTHER
# P                  NATIVE HAWAIIAN or PACIFIC ISLANDER
# U                  UNDESIGNATED
# W                  WHITE
# *************************************************************************** /

# / ***************************************************************************
# Ethnic codes
# ethnicity          description
# *******************************************************************************
# HL                 HISPANIC or LATINO
# NL                 NOT HISPANIC or NOT LATINO
# UN                 UNDESIGNATED
# *************************************************************************** /

['HL+A', 'HL+B', 'HL+I', 'HL+M', 'HL+O', 'HL+W', 'NL+A', 'NL+B', 'NL+I', 'NL+M', 'NL+O', 'NL+W']


In [52]:
#sort into binary classification df
e = pd.DataFrame(a, columns=[f, l])
e['white'] =  z['NL+W_mean'] + z['HL+W_mean']
e['not_white'] = 1 - e['white']
e['race'] = z['race']

race = []

        

for n in list(e['not_white']):
    if n > .50:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)

e['race'] = race
e

,firstname,lastname,white,not_white,race
0,Lynnford,Wilson,0.421815,0.578185,1
1,Jessica,Moore,0.018010,0.981990,1
2,Leonard,Caldwell,0.503121,0.496879,0
3,Scott,Walls,0.504581,0.495419,0
4,Betsy,Kaufman,0.515930,0.484070,0
...,...,...,...,...,...
3427,Arpuro,Monje,NaN,NaN,0
3428,Yuen,Poon,NaN,NaN,0
3429,Eben,Montaquila,NaN,NaN,0
3430,Thaksina,Khaikaew,NaN,NaN,0


In [53]:
print(race.count(1), len(race))
#1294 minority predictions

991 3432


# Package 2: RaceBert

In [54]:
from racebert import RaceBERT
# Load the dataset
namesdf
model = RaceBERT()
# # Concatenate first and last names to form full names
namesdf['fullname'] = namesdf['firstname'] + ' ' + namesdf['lastname']
# # Predict race for all names
# namesdf
predicted_race = model.predict_race(namesdf['fullname'])
# # # # # Add the predicted race to a DataFrame
fd = pd.DataFrame(namesdf,columns =['firstname', 'lastname'])
fd['predicted_race'] = predicted_race

# # # # # Split the ‘predicted_race’ column into ‘label’ and ‘score’ columns
fd[['race', 'probability']] = fd['predicted_race'].apply(pd.Series)
# # # # # Drop the ‘predicted_race’ column
fd.drop('predicted_race', axis=1, inplace=True)
# # # df

In [56]:
#binary sort
# Define a function to calculate the associated probability based on the race
def calculate_probability(row):
    if row['race'] == 'nh_white':
        return 1 - row['probability']
    return row['probability']

# Add a new column "not_white" with the associated probabilities
fd['not_white'] = fd.apply(calculate_probability, axis=1)

fd

,firstname,lastname,race,probability,not_white
0,Lynnford,Wilson,nh_black,0.819709,0.819709
1,Jessica,Moore,nh_white,0.798271,0.201729
2,Leonard,Caldwell,nh_white,0.597929,0.402071
3,Scott,Walls,nh_white,0.976283,0.023717
4,Betsy,Kaufman,nh_white,0.963490,0.036510
...,...,...,...,...,...
18104,Arpuro,Monje,hispanic,0.915842,0.915842
18128,Yuen,Poon,api,0.983733,0.983733
18135,Eben,Montaquila,nh_white,0.967880,0.032120
18141,Thaksina,Khaikaew,api,0.777895,0.777895


# Next:
## Adding probability columns that an owner is not white to the original data frame

In [59]:
# mergent_true_df['prob_nw_eth_wiki'] = a['not_white']
# mergent_true_df['prob_nw_eth_fl_reg_name'] = b['not_white']
# mergent_true_df['prob_nw_eth_fl_five_cat'] = c['not_white']
# mergent_true_df['prob_nw_eth_census_fn'] = d['not_white']
# mergent_true_df['prob_nw_eth_census_ln'] = dd['not_white']
# mergent_true_df['prob_nw_eth_nc'] = e['not_white']
mergent_true_df['prob_nw_rbert']=fd['not_white']
#18156 - 6356
# nonull = mergent_true_df. dropna() 
# nonull
mergent_true_df.to_csv('../data/mergent_and_library/final_data_v2.csv')
mergent_true_df

,Unnamed: 0,duns,company_name,firstname,lastname,title,gender,flag_owner,flag_presi_vicepresi,prob_nw_eth_wiki,...,naics2,naics_name,flag_executive_reported,geoid,prop_nwh,geometry,prob_spacy,prob_gt,prob_ld,prob_nw_rbert
0,0,00-138-3244,"WILSON, LYNNFORD SAMUEL JR",Lynnford,Wilson,Owner,U,1,0,0.917946,...,62,Health Care and Social Assistance,1,5.105948e+10,26.602177,"c(-77.324596, 38.999535)",0.999995,0.0,1,0.819709
1,1,00-145-3948,CROCODILE CONSULTING,Jessica,Moore,Administrative Assistant Sales,FEMALE,0,0,NaN,...,54,"Professional, Scientific, and Technical Services",1,5.105948e+10,20.372268,"c(-77.389534, 38.994397)",0.000000,0.0,0,0.201729
2,2,00-145-3948,CROCODILE CONSULTING,Leonard,Caldwell,Owner,MALE,1,0,NaN,...,54,"Professional, Scientific, and Technical Services",1,5.105948e+10,20.372268,"c(-77.389534, 38.994397)",0.000000,0.0,0,0.402071
3,3,00-171-0412,CENTREVILLE TAX SERVICE LLC,Scott,Walls,Owner,MALE,1,0,0.087375,...,54,"Professional, Scientific, and Technical Services",1,5.105949e+10,19.153605,"c(-77.460635, 38.8639)",0.999994,0.0,1,0.023717
4,4,00-183-0152,KAUFMAN ASSOC,Betsy,Kaufman,Owner,FEMALE,1,0,0.957856,...,54,"Professional, Scientific, and Technical Services",1,5.105942e+10,36.208236,"c(-77.08656, 38.777228)",0.999673,0.0,1,0.036510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18151,18151,96-994-4219,"GEOPAY, INC.",Karla,Russell,Chief Operating Officer,FEMALE,0,0,0.747244,...,81,Other Services (except Public Administration),1,5.105948e+10,34.302699,"c(-77.331479, 38.948677)",0.000000,0.0,0,NaN
18152,18152,96-995-4812,K EDWARD ROBINSON & ASSOCIATES LLC,Keith,Robinson,Pres-ceo,MALE,0,0,NaN,...,54,"Professional, Scientific, and Technical Services",1,5.105942e+10,62.402930,"c(-77.230478, 38.707725)",0.999673,0.0,1,NaN
18153,18153,96-996-0488,"LAGNIAPPE CONSULTING, LLC",Wallace,Daniels,Owner,MALE,1,0,0.064103,...,54,"Professional, Scientific, and Technical Services",1,5.105942e+10,60.324061,"c(-77.097715, 38.7433)",0.000000,0.0,0,NaN
18154,18154,96-996-0488,"LAGNIAPPE CONSULTING, LLC",Omar,Daniels,Co-owner,MALE,0,0,0.022954,...,54,"Professional, Scientific, and Technical Services",1,5.105942e+10,60.324061,"c(-77.097715, 38.7433)",0.000000,0.0,0,NaN
